In [2]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [3]:
%%time
train = pd.read_csv("../data/train.csv", dtype=dtypes)

CPU times: user 1min 50s, sys: 5.98 s, total: 1min 56s
Wall time: 1min 56s


In [4]:
%%time
test = pd.read_csv("../data/test.csv", dtype=dtypes)

CPU times: user 11.9 s, sys: 196 ms, total: 12.1 s
Wall time: 12.1 s


In [5]:
%%time
click_data = pd.concat([train, test])

CPU times: user 7.42 s, sys: 4.08 s, total: 11.5 s
Wall time: 11.5 s


In [6]:
%%time
click_data.sort_values(by=["click_time", "is_attributed"], inplace=True)

CPU times: user 34.4 s, sys: 14.2 s, total: 48.6 s
Wall time: 48.5 s


In [7]:
%%time
click_data.reset_index(inplace=True)

CPU times: user 316 ms, sys: 404 ms, total: 720 ms
Wall time: 717 ms


In [20]:
%%time
click_time = click_data[["index", "click_time"]]

CPU times: user 1.59 s, sys: 796 ms, total: 2.38 s
Wall time: 2.37 s


In [23]:
%%time
click_data.drop(["click_time"], axis=1, inplace=True)

CPU times: user 3.9 s, sys: 2.74 s, total: 6.64 s
Wall time: 6.64 s


In [25]:
%%time
click_data.to_hdf("../data/click_data.hdf", "click_data")

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['attributed_time']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


CPU times: user 14.7 s, sys: 11.3 s, total: 26 s
Wall time: 25.9 s


In [26]:
%%time
click_time.to_hdf("../data/click_time.hdf", "click_time")

CPU times: user 9.37 s, sys: 4.77 s, total: 14.1 s
Wall time: 14.1 s


In [3]:
%%time
click_data = pd.read_hdf("../data/click_data.hdf", "click_data")

CPU times: user 12.4 s, sys: 9.44 s, total: 21.9 s
Wall time: 3min 6s


In [4]:
%%time
click_time = pd.read_hdf("../data/click_time.hdf", "click_time")

CPU times: user 6.56 s, sys: 3.82 s, total: 10.4 s
Wall time: 1min 16s


In [6]:
%%time
click_data.set_index("index", inplace=True)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 6.3 ms


In [7]:
%%time
click_time.set_index("index", inplace=True)

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 5.97 ms


In [23]:
%%time
click_data = pd.concat([click_data, click_time], axis=1)

CPU times: user 7.08 s, sys: 3.98 s, total: 11.1 s
Wall time: 11.1 s


In [24]:
del click_time
gc.collect()

5711

In [25]:
click_data

,app,attributed_time,channel,click_id,device,ip,is_attributed,os,click_time
index,,,,,,,,,
0,3,NaN,379,NaN,1,83230,0.0,13,2017-11-06 14:32:21
1,3,NaN,379,NaN,1,17357,0.0,19,2017-11-06 14:33:34
2,3,NaN,379,NaN,1,35810,0.0,13,2017-11-06 14:34:12
3,14,NaN,478,NaN,1,45745,0.0,13,2017-11-06 14:34:52
4,3,NaN,379,NaN,1,161007,0.0,13,2017-11-06 14:35:08
5,3,NaN,379,NaN,1,18787,0.0,16,2017-11-06 14:36:26
6,3,NaN,379,NaN,1,103022,0.0,23,2017-11-06 14:37:44
7,3,NaN,379,NaN,1,114221,0.0,19,2017-11-06 14:37:59
8,3,NaN,379,NaN,1,165970,0.0,13,2017-11-06 14:38:10
